In [100]:
import pandas as pd
import numpy as np
import puffin.tardis_importer as ti
from concurrent.futures import ProcessPoolExecutor
from pathos.pools import ParallelPool
import sys
sys.path.append('../tests/')
import tardis_tester as tt
import math

In [2]:
ti.batch_download(exchange='binance-futures', symbol_list=['BTCUSDT'], start_date='2021-10-14', end_date = '2021-10-15', data_types =['quotes', 'trades'])

In [3]:
ti_trades = ti.generate_trade_data_from_gzip('BTCUSDT', './datasets/binance-futures_trades_2021-10-14_BTCUSDT.csv.gz')
ti_trades.to_csv('./datasets/binance-futures_trades_2021-10-14_BTCUSDT.csv', index = False)

ti_tob = ti.generate_tob_data_from_gzip('BTCUSDT', './datasets/binance-futures_quotes_2021-10-14_BTCUSDT.csv.gz')
ti_tob.to_csv('./datasets/binance-futures_quotes_2021-10-14_BTCUSDT.csv', index = False)

In [16]:
df_tob = pd.read_parquet('/Users/rahmanw/Downloads/Sandbox/20211014_4627_tob.parquet')
df_trades = pd.read_parquet('/Users/rahmanw/Downloads/Sandbox/20211014_4627_trade.parquet')
print(len(df_trades))

1550910


In [20]:
#reduce the intersection to just where the trade id's overlap (we collect a bit of extra date on either side of midnight)
first_id = ti_trades['trade_id'].iloc[0]
last_id = ti_trades['trade_id'].iloc[-1]
print(first_id, last_id)
df_trades = df_trades[(df_trades['ignore_first_trade_id'] >= first_id) & (df_trades['ignore_last_trade_id'] <= last_id)]
print(len(df_trades))

1536569832 1539609453
1548314


In [18]:
df_trades

,trade_date_nanos,instrument_id,receive_timestamp_nanos,exchange_timestamp_nanos,price,qty,buyer_is_market_maker,ignore_first_trade_id,ignore_last_trade_id
2594,1634169600000000000,4627,1634169608820374457,1634169602475000000,57393.65,0.005,True,1536569832,1536569832
2595,1634169600000000000,4627,1634169611487457032,1634169602972000000,57393.65,1.027,True,1536569833,1536569846
2596,1634169600000000000,4627,1634169611646244133,1634169602990000000,57393.66,0.198,False,1536569847,1536569856
2597,1634169600000000000,4627,1634169611678412326,1634169603078000000,57393.65,0.141,True,1536569857,1536569857
2598,1634169600000000000,4627,1634169611688295673,1634169603093000000,57393.66,1.156,False,1536569858,1536569862
...,...,...,...,...,...,...,...,...,...
1550903,1634169600000000000,4627,1634255999362236704,1634255999204000000,57379.86,0.001,False,1539609445,1539609445
1550904,1634169600000000000,4627,1634255999642112559,1634255999481000000,57379.86,0.021,False,1539609446,1539609446
1550905,1634169600000000000,4627,1634255999909614965,1634255999899000000,57379.85,0.009,True,1539609447,1539609450
1550906,1634169600000000000,4627,1634255999955239842,1634255999899000000,57377.57,0.071,True,1539609451,1539609451


In [19]:
ti_trades

,instrument_id,receive_timestamp_nanos,exchange_timestamp_nanos,price,qty,buyer_is_market_maker,trade_id
0,4627,1634169608672425984,1634169602475000064,57393.65,0.005,True,1536569832
1,4627,1634169611430328064,1634169602972000000,57393.65,0.001,True,1536569833
2,4627,1634169611466356992,1634169602976000000,57393.65,0.050,True,1536569834
3,4627,1634169611472185088,1634169602982000128,57393.65,0.185,True,1536569835
4,4627,1634169611472370944,1634169602982000128,57393.65,0.002,True,1536569836
...,...,...,...,...,...,...,...
3039500,4627,1634255999912271104,1634255999899000064,57379.85,0.002,True,1539609449
3039501,4627,1634255999912512000,1634255999899000064,57379.85,0.002,True,1539609450
3039502,4627,1634255999913005056,1634255999899000064,57377.57,0.071,True,1539609451
3039503,4627,1634255999960958976,1634255999947000064,57378.20,0.082,True,1539609452


In [147]:
mg = ti_trades.merge(df_trades, how='left', left_on ='trade_id', right_on='ignore_first_trade_id')
mg = mg[["instrument_id_x", "receive_timestamp_nanos_x", "exchange_timestamp_nanos_x", "price_x", "qty_x", "buyer_is_market_maker_x", "trade_id", "ignore_first_trade_id"]]
mg['ignore_first_trade_id'] = mg['ignore_first_trade_id'].fillna(method='ffill')
mg['ignore_first_trade_id'] = mg['ignore_first_trade_id'].astype(int)
grouped = mg.groupby(by='ignore_first_trade_id').sum()
grouped = grouped.iloc[:-1]
df_trades = df_trades.iloc[:-1]
grouped

,instrument_id_x,receive_timestamp_nanos_x,exchange_timestamp_nanos_x,price_x,qty_x,buyer_is_market_maker_x,trade_id
ignore_first_trade_id,,,,,,,
1536569832,4627.0,1.634170e+18,1.634170e+18,57393.65,0.005,1,1.536570e+09
1536569833,64778.0,2.287837e+19,2.287837e+19,803511.10,1.027,14,2.151198e+10
1536569847,46270.0,1.634170e+19,1.634170e+19,573936.60,0.198,0,1.536570e+10
1536569857,4627.0,1.634170e+18,1.634170e+18,57393.65,0.141,1,1.536570e+09
1536569858,23135.0,8.170848e+18,8.170848e+18,286968.30,1.156,0,7.682849e+09
...,...,...,...,...,...,...,...
1539609444,4627.0,1.634256e+18,1.634256e+18,57380.00,0.102,0,1.539609e+09
1539609445,4627.0,1.634256e+18,1.634256e+18,57379.86,0.001,0,1.539609e+09
1539609446,4627.0,1.634256e+18,1.634256e+18,57379.86,0.021,0,1.539609e+09


In [148]:
df_trades

,trade_date_nanos,instrument_id,receive_timestamp_nanos,exchange_timestamp_nanos,price,qty,buyer_is_market_maker,ignore_first_trade_id,ignore_last_trade_id
2594,1634169600000000000,4627,1634169608820374457,1634169602475000000,57393.65,0.005,True,1536569832,1536569832
2595,1634169600000000000,4627,1634169611487457032,1634169602972000000,57393.65,1.027,True,1536569833,1536569846
2596,1634169600000000000,4627,1634169611646244133,1634169602990000000,57393.66,0.198,False,1536569847,1536569856
2597,1634169600000000000,4627,1634169611678412326,1634169603078000000,57393.65,0.141,True,1536569857,1536569857
2598,1634169600000000000,4627,1634169611688295673,1634169603093000000,57393.66,1.156,False,1536569858,1536569862
...,...,...,...,...,...,...,...,...,...
1550902,1634169600000000000,4627,1634255999111097596,1634255998952000000,57380.00,0.102,False,1539609444,1539609444
1550903,1634169600000000000,4627,1634255999362236704,1634255999204000000,57379.86,0.001,False,1539609445,1539609445
1550904,1634169600000000000,4627,1634255999642112559,1634255999481000000,57379.86,0.021,False,1539609446,1539609446
1550905,1634169600000000000,4627,1634255999909614965,1634255999899000000,57379.85,0.009,True,1539609447,1539609450


In [152]:
diff = grouped['qty_x'].apply(lambda x : float('{:.3f}'.format(x))).values  - df_trades['qty'].apply(lambda x : float('{:.3f}'.format(x))).values
np.where(diff > 0)

(array([  21719,   40254,  206337,  238969,  259211,  261936,  297006,
         474483,  684530,  823114,  951030,  983402, 1065008, 1113666,
        1380974]),)

In [87]:
def compare_data(df_us, df_tardis):
    agg_prices = []
    agg_qtys = []
    for i in range(len(df_us)):
        start_id = df_us.iloc[i]['ignore_first_trade_id']
        end_id = df_us.iloc[i]['ignore_last_trade_id']
        prices = df_tardis['price'][(df_tardis['trade_id'] >= start_id) & (df_tardis['trade_id'] <= end_id)].values
        agg_prices.append(prices)
        agg_qty = df_tardis['qty'][(df_tardis['trade_id'] >= start_id) & (df_tardis['trade_id'] <= end_id)].sum()
        agg_qty = float('{:,.3f}'.format(agg_qty))
        agg_qtys.append(agg_qty)
    errors = {'unique_price_errors':[], 'price_mismatch_error':[], 'data':df_us}
    prices = []
    for i in range(len(agg_prices)):
        unique_prices = np.unique(agg_prices[i])
        if len(unique_prices) == 1:
            prices.append(float('{:.3f}'.format(unique_prices[0])))
        else:
            prices.append(float('{:.3f}'.format(unique_prices[0])))
            errors['unique_price_errors'].append(i)
            
    diff_prices = df_us['price'] == prices
    diff_qtys = df_us['qty'] == agg_qtys
    print('differences in prices:', sum(diff_prices) - len(df_us))
    print('differences in qtys:', sum(diff_qtys) - len(df_us))
    return errors
        

In [88]:
threads = 16
steps = math.ceil(len(df_trades)/16)

In [101]:
results = []
with ParallelPool(threads) as pool:
        datasets = [df_trades.iloc[i*steps:(i + 1)*steps] for i in range(threads)] 
        for res in pool.map(compare_data, datasets, [ti_trades]*threads):
            results.append(res)
        #for i in range(threads):
        #    res = pool.submit(tt.compare_data, datasets[i], ti_trades)
        #    results.append(res)

SyntaxError: EOL while scanning string literal (<string>, line 1)

In [96]:
results

[<Future at 0x7f9a2d031be0 state=finished raised BrokenProcessPool>,
 <Future at 0x7f9a2d01f970 state=finished raised BrokenProcessPool>,
 <Future at 0x7f9a98664040 state=finished raised BrokenProcessPool>,
 <Future at 0x7f9a2d013460 state=finished raised BrokenProcessPool>,
 <Future at 0x7f9a2d013a60 state=finished raised BrokenProcessPool>,
 <Future at 0x7f9a2d02ad60 state=finished raised BrokenProcessPool>,
 <Future at 0x7f9a2d015970 state=finished raised BrokenProcessPool>,
 <Future at 0x7f9a2d00eb80 state=finished raised BrokenProcessPool>,
 <Future at 0x7f9a2d015a60 state=finished raised BrokenProcessPool>,
 <Future at 0x7f9a2d0153a0 state=finished raised BrokenProcessPool>,
 <Future at 0x7f9a2d015a90 state=finished raised BrokenProcessPool>,
 <Future at 0x7f9a2d031c10 state=finished raised BrokenProcessPool>,
 <Future at 0x7f9a2dbad370 state=finished raised BrokenProcessPool>,
 <Future at 0x7f9a2dbad160 state=finished raised BrokenProcessPool>,
 <Future at 0x7f9a2dbad430 state=f